# Data Preparation with Manual Feature Selection*

*Features are selected based the analysis in (preprocess_feature_analysis) 


*Feature selection is specific for OpenFace dataset (not forMediapipe)

In [1]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime

from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
import scipy
from scipy import stats

In [12]:
# path_daisee = ('../../Dataset/DAiSEE/OpenFace_2.2.0_win_x64/processed/csv/')
path_daisee = ('../../Dataset/DAiSEE/')
# path_emotiw = ('../../Dataset/EmotiW/Engagement recognition/01. Train and validation/')
df_feat = pd.read_csv(path_daisee+'CSV_Mediapipe/Train/181374016.csv') #Load a feature sample
print(df_feat.shape)

# Based on features of OpenFace2.0 
features31 = [5,6,7,8,9,10,11,12,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315]
features311 = list(range(5,316))

# REMOVE SPACES FROM COLUMN NAMES
df = pd.DataFrame(df_feat)
df.columns = df.columns.str.replace(' ','')

#for num, col_name in enumerate(df.columns):
#    print(num, col_name)

(300, 2004)


Single Test

In [4]:
# # Only for Features from OpenFace2.0
# # SELECT DATA from TIMESTEPS between 0.5 to 9.5 
# dfqr = df.query('(timestamp > 0.3) & (timestamp < 9.9)')
# # print(dfqr)

# # SELECT 31 FEATURES COLUMNS
# dfqr = dfqr.iloc[:,features311]
# print(dfqr.shape)

# For Mediapipe
# ====================================
dfqr = df

In [5]:
#MOVING AVERAGE using PANDAS
by = 5 #stride
win = 15 #windows
start = 0

# IMPUTATION (replace the missing or null values with particular value in the entire dataset)
imputer = SimpleImputer(strategy='median')
dfqr = pd.DataFrame(imputer.fit_transform(dfqr))
dfqr.columns = dfqr.columns

# MEAN
dfmean = dfqr.rolling(win).mean()[start::by] 
dfmean = dfmean.dropna(inplace=False)
print(dfmean.shape)

# MIN
dfmin = dfqr.rolling(win).min()[start::by]
dfmin = dfmin.dropna(inplace=False)
print(dfmin.shape)

# MAX
dfmax = dfqr.rolling(win).max()[start::by] 
dfmax = dfmax.dropna(inplace=False)
print(dfmax.shape)

# STD
dfstd = dfqr.rolling(win).std()[start::by]
dfstd = dfstd.dropna(inplace=False)
print(dfstd.shape)

# KURTOSIS
# dfkurt = dfqr.rolling(win).kurt()[start::by]
# dfkurt = dfkurt.iloc[3:,:]
#dfkurt = dfkurt.fillna(0)
# print(dfkurt.shape)

# SKEWNESS
# dfskew = dfqr.rolling(win).skew()[start::by]
# dfskew = dfskew.iloc[3:,:]
#dfskew = dfskew.fillna(0)
#print(dfskew.shape)

# MEDIAN
dfmed = dfqr.rolling(win).median()[start::by]
dfmed = dfmed.dropna()
print(dfmed.shape)

# CORRELATION
dfcorr = dfqr.rolling(win).median()[start::by]
dfcorr = dfcorr.dropna()
print(dfcorr.shape)

# NEW FEATURES
dfnew = pd.concat([dfmean,dfmin,dfmax,dfstd, dfmed,dfcorr], axis=1) #,dfkurt,dfskew
print(dfnew.shape)

(57, 2004)
(57, 2004)
(57, 2004)
(57, 2004)
(57, 2004)
(57, 2004)
(57, 12024)


In [6]:
rows_with_nan = []
for  index, row in dfnew.iterrows():
    is_nan_series = row.isnull()
    if is_nan_series.any():
        rows_with_nan.append(index)
print(rows_with_nan)

[]


In [7]:
# Check Missing data
missing_data = dfnew.isnull()
print(missing_data)

# Correct data format
print(dfnew.dtypes)

for column in missing_data.columns.values.tolist():
   print(column)
#    print (missing_data[column].value_counts())
   print('x')

      0      1      2      3      4      5      6      7      8      9     \
15   False  False  False  False  False  False  False  False  False  False   
20   False  False  False  False  False  False  False  False  False  False   
25   False  False  False  False  False  False  False  False  False  False   
30   False  False  False  False  False  False  False  False  False  False   
35   False  False  False  False  False  False  False  False  False  False   
40   False  False  False  False  False  False  False  False  False  False   
45   False  False  False  False  False  False  False  False  False  False   
50   False  False  False  False  False  False  False  False  False  False   
55   False  False  False  False  False  False  False  False  False  False   
60   False  False  False  False  False  False  False  False  False  False   
65   False  False  False  False  False  False  False  False  False  False   
70   False  False  False  False  False  False  False  False  False  False   

## DAISEE Extraction

In [14]:
# def extract(datapath,labelfile,colnum): # for OpenFace dataset 
def extract(datapath,labelfile): #for Mediapipe dataset
    df = pd.read_csv(labelfile) #load label
    
    #create empty list
    data_list = []
    label_list = []

    by = 10 #stride
    win = 20 #windows
    start = 0
    
    for noid, clipid in enumerate(df.New_ClipID):
        temp_df = pd.read_csv(datapath+clipid) 
        # print(temp_df)

        # REMOVE SPACES FROM COLUMN NAMES
        temp_df = pd.DataFrame(df_feat)
        temp_df.columns = temp_df.columns.str.replace(' ','')
        # IMPUTATION
        imputer = SimpleImputer(strategy='median')
        imputed_df = pd.DataFrame(imputer.fit_transform(temp_df))
        imputed_df.columns = temp_df.columns
        # SELECT DATA from TIMESTEPS between 0.5 to 9.5 (Only for OpenFace)
        # temp_df = temp_df.query('(timestamp > 0.0) & (timestamp < 10)')
        # temp_df = temp_df.iloc[:,colnum] #select features 

        # MOVING AVERAGE (Mean)
        dfmean = temp_df.rolling(win).mean()[start::by] 
        dfmean = dfmean.dropna(inplace=False)
        #print(dfmean.shape)
        # MIN
        dfmin = temp_df.rolling(win).min()[start::by]
        dfmin = dfmin.dropna(inplace=False)
        #print(dfmin.shape)
        # MAX
        dfmax = temp_df.rolling(win).max()[start::by] 
        dfmax = dfmax.dropna(inplace=False)
        #print(dfmax.shape)
        # STD
        dfstd = temp_df.rolling(win).std()[start::by]
        dfstd = dfstd.dropna(inplace=False)
        #print(dfstd.shape)
        # KURTOSIS
        # dfkurt = temp_df.rolling(win).kurt()[start::by]
        # dfkurt = dfkurt.iloc[3:,:]
        # dfkurt = dfkurt.fillna(0)
        # print(dfkurt.shape)
        # SKEWNESS
        # dfskew = temp_df.rolling(win).skew()[start::by]
        # dfskew = dfskew.iloc[3:,:]
        #dfskew = dfskew.fillna(0)
        # print(dfskew.shape)
        # MEDIAN
        dfmed = temp_df.rolling(win).median()[start::by]
        dfmed = dfmed.dropna(inplace=False)
        #print(dfmed.shape)
        # CORRELATION
        dfcorr = temp_df.rolling(win).median()[start::by]
        dfcorr = dfcorr.dropna(inplace=False)
        #print(dfcorr.shape)

        # NEW FEATURES
        dfnew = pd.concat([dfmean,dfmin,dfmax,dfstd,dfmed,dfcorr], axis=1)
        #print(dfnew.shape)
        #print(dfnew)

        #imputed = pd.DataFrame(imputer.fit_transform(dfnew))
        #print(imputed)
        #print(imputed)
        #dfnew.columns = imputed.columns
        #imputed.columns = dfnew.columns
        #print(imputed.print)

        data_list.append(dfnew)
        labl = df.Engagement[noid] 
        labl = np.array(labl)
        label_list.append(labl)    
    
    X = np.array(data_list) #features
    Y = np.array(label_list) #labels
    print('X shape:{}, Y shape:{}'.format(X.shape,Y.shape))
    
    return X, Y

In [15]:
tic = datetime.now()
X_train, Y_train = extract(path_daisee+'CSV_Mediapipe/Train/',path_daisee+'/labels/TrainLabels.csv') #,features311
X_val, Y_val = extract(path_daisee+'CSV_Mediapipe/Validation/',path_daisee+'/labels/ValidationLabels.csv') #,features311
X_test, Y_test = extract(path_daisee+'CSV_Mediapipe/Test/',path_daisee+'/labels/TestLabels.csv') #,features311
time = datetime.now() - tic
print('Processing time {}'.format(time))

X shape:(5467, 28, 12024), Y shape:(5467,)
X shape:(1703, 28, 12024), Y shape:(1703,)
X shape:(1784, 28, 12024), Y shape:(1784,)
Processing time 3:17:40.167217


In [19]:
# SAVE INPUT
with open('daisee_Mediapipe_57timesteps.pkl','wb') as f:
    pickle.dump([X_train, X_val, X_test, Y_train, Y_val, Y_test], f) #

In [20]:
# LOAD INPUT
with open('daisee_Mediapipe_57timesteps.pkl','rb') as f:
    X_train, X_val, X_test, y_train, y_val, y_test = pickle.load(f)

## Averaged Data Extraction

In [28]:
def averaged(set,datapath,labelfile):
    df = pd.read_csv(labelfile) #load label

    feat = []
    for noid, clipid in enumerate(df.New_ClipID):
        features = pd.read_csv(datapath+clipid, index_col=None, header=0)
        # features = features.drop(features.columns[0:5], axis=1) #Only for OpenFace data
        features = features.mean(axis=0) #For average all data
        features = features.dropna(inplace=False)
        # features = features.groupby(np.arange(len(features))//30).mean() #average every 30 frames, resulting 10 timesteps

        #add label in column
        label = df.Engagement[noid]
        features['Engagement'] = label
        features = pd.DataFrame(features).T #transpose after averaging
    #     print(features)
        feat.append(features)
    #     print(feat)
    #     break
        
    average = pd.concat(feat, axis=0, ignore_index=True)
    average.to_csv('average_'+set+'_mediapipe.csv', header=True)
    # print(average)
    return


In [29]:
tic = datetime.now()
train = averaged('train',path_daisee+'CSV_Mediapipe/Train/',path_daisee+'labels/TrainLabels.csv')
val = averaged('val',path_daisee+'CSV_Mediapipe/Validation/',path_daisee+'labels/ValidationLabels.csv')
test = averaged('test',path_daisee+'CSV_Mediapipe/Test/',path_daisee+'labels/TestLabels.csv')
time = datetime.now() - tic
print('Processing time {}'.format(time))

Processing time 0:18:12.141233


## EMOTIW 2018

In [ ]:
def load_emotiw2018(datapath,colnum):
    df = pd.read_csv(path_emotiw+'Labels_Engagement.csv') #load label
    id = df.iloc[:,0]
    
    #create empty list
    data_list = []
    label_list = []

    by = 5 #stride
    win = 10 #windows
    start = 0
    
    for noid, clipid in enumerate(id):
        temp_df = pd.read_csv(datapath+clipid) 
        # print(noid, clipid)
        # print(temp_df)

        # REMOVE SPACES FROM COLUMN NAMES
        temp_df = pd.DataFrame(df_feat)
        temp_df.columns = temp_df.columns.str.replace(' ','')
        # IMPUTATION
        imputer = SimpleImputer(strategy='median')
        imputed_df = pd.DataFrame(imputer.fit_transform(temp_df))
        imputed_df.columns = temp_df.columns
        # SELECT DATA from TIMESTEPS between 30 seconds to 3 minutes 30 secs (210 seconds)
        # temp_df = temp_df.query('(timestamp < 200)') #for downsampled
        temp_df = temp_df.query('(timestamp > 0.30) & (timestamp < 210)')
        temp_df = temp_df.iloc[:,colnum] #select features 

        # MOVING AVERAGE (Mean)
        dfmean = temp_df.rolling(win).mean()[start::by] 
        dfmean = dfmean.dropna(inplace=False)
        # MIN
        dfmin = temp_df.rolling(win).min()[start::by]
        dfmin = dfmin.dropna(inplace=False)
        # MAX
        dfmax = temp_df.rolling(win).max()[start::by] 
        dfmax = dfmax.dropna(inplace=False)
        # STD
        dfstd = temp_df.rolling(win).std()[start::by]
        dfstd = dfstd.dropna(inplace=False)
        # # KURTOSIS
        # dfkurt = temp_df.rolling(win).kurt()[start::by]
        # dfkurt = dfkurt.iloc[3:,:]
        # dfkurt = dfkurt.fillna(0)
        # # SKEWNESS
        # dfskew = temp_df.rolling(win).skew()[start::by]
        # dfskew = dfskew.iloc[3:,:]
        # dfskew = dfskew.fillna(0)
        # MEDIAN
        dfmed = temp_df.rolling(win).median()[start::by]
        dfmed = dfmed.dropna(inplace=False)
        # CORRELATION
        dfcorr = temp_df.rolling(win).median()[start::by]
        dfcorr = dfcorr.dropna(inplace=False)

        # NEW FEATURES
        dfnew = pd.concat([dfmean,dfmin,dfmax,dfstd,dfmed,dfcorr], axis=1)
        # ,dfkurt,dfskew,

        temp_df = np.array(dfnew)
        data_list.append(temp_df)
        labl = df.iloc[noid,1]  
        labl = np.array(labl)
        label_list.append(labl)    
    
    X = np.array(data_list) #features
    Y = np.array(label_list) #labels
    print('X shape:{}, Y shape:{}'.format(X.shape,Y.shape))
    
    return X, Y

In [ ]:
# EXTRACT EMOTIW2018
tic = datetime.now()
# X, y = load_emotiw2018(path_emotiw+'trainval/',features31)
#X, y = load_emotiw2018('C:/Users/hasegawa-lab-pc/OneDrive - Japan Advanced Institute of Science and Technology/Documents/Dataset/EmotiW/Downsampled video/csv/',features31)
X, y = load_emotiw2018(path_emotiw+'trainval/',features31)
time = datetime.now() - tic
print('Processing time {}'.format(time))

In [ ]:
# SAVE INPUT
with open('extracted\emotiw_186.pkl','wb') as f:
    pickle.dump([X, y], f) #

In [ ]:
# LOAD INPUT
with open('extracted\emotiw_31_56.pkl','rb') as f:
    X, y = pickle.load(f)

## PCA

In [ ]:
print(X_train.shape,X_val.shape,X_test.shape)
print(X.shape)

In [ ]:
# RESHAPE DATA
# create one big data panel with n_features series
X_reshaped = X.reshape((X.shape[0]*X.shape[1], X.shape[2])) 

In [ ]:
#  PCA
n_comp = 20
pca = PCA(n_components=n_comp)
pca.fit(X_reshaped)

In [ ]:
X_transformed = np.empty([X.shape[0],X.shape[1],n_comp])
# iteratively apply the transformation to each instance of the original dataset
for i in range(len(X)):
    X_transformed[i]=pca.transform(X[i])

In [ ]:
print(X_transformed.shape)